# AML Homework 2

## Programming Exercise

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qu4ntum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/qu4ntum/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/qu4ntum/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### (a)

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print("Training data have " + str(train.shape[0]) + " data points.")
print("Testing data have " + str(test.shape[0]) + " data points.")

num_fake = train['target'].value_counts(dropna=False)[0]
num_real = train['target'].value_counts(dropna=False)[1]

print("Percentage of real disasters: " + str(round(num_real/(num_real+num_fake),3)))
print("Percentage of not real disasters: " + str(round(num_fake/(num_real+num_fake),3)))

Training data have 7613 data points.
Testing data have 3263 data points.
Percentage of real disasters: 0.43
Percentage of not real disasters: 0.57


### (b)

In [3]:
x = train.iloc[:,:-1]
y = train.iloc[:,-1:]
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.30)

### (c)

In [4]:
def preprocess(text):
    stopword = stopwords.words('english')
    wordnet_lemmatizer = WordNetLemmatizer()
    #Tokenize text
    word_tokens = nltk.word_tokenize(text)
    #Convert to lowercase
    word_tokens = [w.lower() for w in word_tokens]
    #Lemmatization and stripping stopwords
    word_tokens = [wordnet_lemmatizer.lemmatize(w) for w in word_tokens]
    #Strip punctuation,@,and urls
    word_tokens = [w for w in word_tokens if w not in punctuation]
    return word_tokens


tmp = []
for text in train['text']:
    tmp.append(preprocess(text))
train['text'] = tmp

tmp = []
for text in test['text']:
    tmp.append(preprocess(text))
test['text'] = tmp

x = train.iloc[:,:-1]
y = train.iloc[:,-1:]
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.30)


### (d)

In [5]:
def token_to_sentence(df):#Convert the tokenized 'text' field back to sentence in order to be vectorized
    tmp = []
    for sentence in df['text']:
        str = ''
        for token in sentence:
            str += token + ' '
        tmp.append(str)
    return tmp
        
train_text = token_to_sentence(x_train)
dev_text = token_to_sentence(x_dev)
test_text = token_to_sentence(test)


x_train['text'] = train_text
x_dev['text'] = dev_text
test['text'] = test_text

# vectorizer = CountVectorizer(binary=True, min_df=10)
# vectorizer.fit(x_train["text"])

# x_train_bow = vectorizer.transform(x_train['text'])
# x_dev_bow = vectorizer.transform(x_dev['text'])

M = 0
max_score = -9999

for i in range(50):
    vectorizer = CountVectorizer(binary=True, min_df=i)
    vectorizer.fit(x_train["text"])#Build a vocabulary of the words appearing in the training set

    x_train_bow = vectorizer.transform(x_train['text'])
    x_dev_bow = vectorizer.transform(x_dev['text'])

    model_logistic = LogisticRegression(penalty='none')
    model_logistic.fit(x_train_bow, y_train)

    y_dev_predict = model_logistic.predict(x_dev_bow)

    if f1_score(y_dev, y_dev_predict) > max_score: 
        max_score = f1_score(y_dev, y_dev_predict)
        M = i

vectorizer = CountVectorizer(binary=True, min_df=M)
vectorizer.fit(x_train["text"])#Build a vocabulary of the words appearing in the training set


x_train_bow = vectorizer.transform(x_train['text'])#Bag of words for training data
x_dev_bow = vectorizer.transform(x_dev['text'])#Bag of words for training data

print("Total number of feature for training set Bag of Words vector: " + str(x_train_bow.len()))
print("Total number of feature for development set Bag of Words vector: " + str(x_dev_bow.len()))


# x_train['text'] = x_train_bow
# x_dev['text'] = x_dev_bow
# test['text'] = test_bow

AttributeError: len not found

In [15]:
print('Total number of features of these vectors (They should be equal since vectorizer is the same): ' + str(x_train_bow.shape[1]))



Total number of features of these vectors (They should be equal since vectorizer is the same): 15988


In [ ]:
# M = 0
# max_score = -9999


# for i in range(100):
#     vectorizer = CountVectorizer(binary=True, min_df=i)
#     vectorizer.fit(x_train["text"])

#     x_train_bow = vectorizer.transform(x_train['text'])
#     x_dev_bow = vectorizer.transform(x_dev['text'])

#     model_logistic = LogisticRegression(penalty='none')
#     model_logistic.fit(x_train_bow, y_train)

#     y_dev_predict = model_logistic.predict(x_dev_bow)

#     if f1_score(y_dev, y_dev_predict) > max_score: 
#         max_score = f1_score(y_dev, y_dev_predict)
#         M = i

# print("Optimum M is: " + str(M))


### (e)

#### i.

In [ ]:
model_logistic = LogisticRegression(penalty='none')
model_logistic.fit(x_train_bow, y_train)

y_train_predict = model_logistic.predict(x_train_bow)
print('F1-Score for training set without regularization: ' + str(f1_score(y_train, y_train_predict)))

y_dev_predict = model_logistic.predict(x_dev_bow)
print('F1-Score for development set without regularization: ' + str(f1_score(y_dev, y_dev_predict)))

#### ii.

In [ ]:
model_logistic_L1 = LogisticRegression(penalty='l1', solver='liblinear')
model_logistic_L1.fit(x_train_bow, y_train)

y_train_predict = model_logistic_L1.predict(x_train_bow)
print('F1-Score for training set with L1 regularization: ' + str(f1_score(y_train, y_train_predict)))

y_dev_predict = model_logistic_L1.predict(x_dev_bow)
print('F1-Score for development set with L1 regularization: ' + str(f1_score(y_dev, y_dev_predict)))

#### iii.

In [ ]:
model_logistic_L2 = LogisticRegression(penalty='l2')
model_logistic_L2.fit(x_train_bow, y_train)

y_train_predict = model_logistic_L2.predict(x_train_bow)
print('F1-Score for training set with L2 regularization: ' + str(f1_score(y_train, y_train_predict)))

y_dev_predict = model_logistic_L2.predict(x_dev_bow)
print('F1-Score for development set with L2 regularization: ' + str(f1_score(y_dev, y_dev_predict)))

#### iv.

In [ ]:
#The logistic classifier without any regularization performed the best on training set.
#The logistic classifier with L2 regularization performed the best on development set.
#Yes there is overfitting across all three logistic classifier since the training set performance is higher than 
#development set performance
#However, regularization does apeart to reduce overfitting since the gap between performance on training and development
#is shrinked after regularization

#### v.

In [ ]:
#Weights
weights = model_logistic_L1.coef_.flatten()

df_weights = pd.DataFrame({'Label': vectorizer.get_feature_names(), 'Weight': weights})
df_weights = df_weights.sort_values(by=['Weight'], ascending=False)
df_weights.head(5)
#Top 3 words: spill, hiroshima, derailment

### (f)

In [ ]:
x_train_2 = x_train_bow.toarray()
y_train_arr = y_train["target"].values

n = x_train_2.shape[0]
d = x_train_2.shape[1]
alpha = 1
K = 2
psis = np.zeros([K,d])
phis = np.zeros([K])

for k in range(K):
    X_k = x_train_2[y_train_arr == k]
    psis[k] = np.mean(X_k, axis=0)
    phis[k] = (X_k.shape[0] + alpha) / (float(n) + alpha * 2)
    
def nb_predictions(x, psis, phis):
    n, d = x.shape
    x = np.reshape(x, (1, n, d))
    psis = np.reshape(psis, (K, 1, d))
    
    psis = psis.clip(1e-14, 1-1e-14)
    
    logpy = np.log(phis).reshape([K,1])
    logpxy = x * np.log(psis) + (1-x) * np.log(1-psis)
    logpyx = logpxy.sum(axis=2) + logpy

    return logpyx.argmax(axis=0).flatten(), logpyx.reshape([K,n])

y_train_predict, logpyx = nb_predictions(x_train_2, psis, phis)
print("F1-score for training set is: " + str(f1_score(y_train, y_train_predict)))
y_dev_predict, logpyx2 = nb_predictions(x_dev_bow.toarray(), psis, phis)
print("F1-score for development set is: " + str(f1_score(y_dev, y_dev_predict)))

### (g)

In [ ]:
#Discriminative classifier (Logistic classifier with L2 regularization) performed the best with development set, but 
#not by much compared to generative classifier. In fact, generative classifier performed better than logistic 
#classifier without regularization

# Generative Model Pros:
# Can do more than just prediction: generation, fill-in missing features, etc.
# Can include extra prior knowledge; if prior knowledge is correct, model will be more accurate.
# Understandability: it is easier to understand the result
# Outlier detection: we may detect via  𝑝(𝑥′)  if  𝑥′  is an outlier.
# Scalability: Simple formulas for maximum likelihood parameters.
# Generation: we can sample  𝑥∼𝑝(𝑥|𝑦)  to generate new data (images, audio)
# Generative Model Cons:
# Generative model assumes independence which is not always true, causing over or under confident of model
# Computationally expensive

# Discriminative model Pros:
# Most state-of-the-art algorithms for classification are discriminative (including neural nets, boosting, SVMs, etc.)
# They are often more accurate because they make fewer modeling assumptions.
# Computationally cheaper
# Discriminative model Cons:
# Cannot generate data 
# Hard to understand




In [ ]:
#Naive bayes model assumes words are uncorrelated.
#Logistic regression requires the observations to be independent of each other
#It is sufficient for bag of words since it is binary.

### (h)

In [ ]:
M = 0
max_score = -9999


for i in range(50):
    vectorizer2 = CountVectorizer(binary=True,min_df=i, ngram_range=(1,2))
    vectorizer2.fit(x_train["text"])

    x_train_bow_2 = vectorizer2.transform(x_train['text'])
    x_dev_bow_2 = vectorizer2.transform(x_dev["text"])

    #Regression with N-gram
    model_logistic_Ngram = LogisticRegression(penalty='l2')
    model_logistic_Ngram.fit(x_train_bow_2, y_train)

    y_dev_predict_Ngram = model_logistic_Ngram.predict(x_dev_bow_2)

    if f1_score(y_dev, y_dev_predict_Ngram) > max_score: 
        max_score = f1_score(y_dev, y_dev_predict_Ngram)
        M = i

#N-Gram models
vectorizer2 = CountVectorizer(binary=True,min_df=M, ngram_range=(1,2))

#Fitting and processing countVectorizer
vectorizer2.fit(x_train["text"])
x_train_bow_2 = vectorizer2.transform(x_train["text"])
x_dev_bow_2 = vectorizer2.transform(x_dev["text"])

dict = vectorizer2.get_feature_names()
count_1gram = 0
count_2gram = 0
for word in dict:
    if ' ' in word:
        count_2gram += 1
    else:
        count_1gram += 1
        
print("Number of 1 grams: " + str(count_1gram))
print("Number of 2 grams: " + str(count_2gram))

tmp = []
for word in dict[1000:]: #some vocabs in the beginning contains numbers so I skipped those
    if ' ' in word:
        tmp.append(word)
    if len(tmp)==10:
        break
        
print("Some 2 grams in the vocabulary: ")
print(tmp)



#N-Gram models
vectorizer3 = CountVectorizer(binary=True,min_df=M, ngram_range=(2,2))#2 gram
vectorizer3.fit(x_train["text"])
x_train_bow_3 = vectorizer3.transform(x_train["text"])
x_dev_bow_3 = vectorizer3.transform(x_dev["text"])

#Regression with N-gram
model_logistic_Ngram = LogisticRegression(penalty='l2')
model_logistic_Ngram.fit(x_train_bow_3, y_train)

#F-1 score with l2 regularization
y_train_predict_Ngram = model_logistic_Ngram.predict(x_train_bow_3)
print("F1-score for training set with L2 Regularization is: " + str(f1_score(y_train, y_train_predict_Ngram)))
y_dev_predict_Ngram = model_logistic_Ngram.predict(x_dev_bow_3)
print("F1-score for development set with L2 Regularization is: " + str(f1_score(y_dev, y_dev_predict_Ngram)))


x_train_3 = x_train_bow_3.toarray()

n = x_train_3.shape[0]
d = x_train_3.shape[1]
alpha = 1
K = 2
psis = np.zeros([K,d])
phis = np.zeros([K])

for k in range(K):
    X_k = x_train_3[y_train_arr == k]
    psis[k] = np.mean(X_k, axis=0)
    phis[k] = (X_k.shape[0] + alpha) / (float(n) + alpha * 2)
    
y_train_predict, logpyx3 = nb_predictions(x_train_3, psis, phis)
print("F1-score for training set is: " + str(f1_score(y_train, y_train_predict)))
y_dev_predict, logpyx4 = nb_predictions(x_dev_bow_3.toarray(), psis, phis)
print("F1-score for development set is: " + str(f1_score(y_dev, y_dev_predict)))

In [ ]:
# M = 0
# max_score = -9999


# for i in range(50):
#     vectorizer2 = CountVectorizer(binary=True,min_df=i, ngram_range=(1,2))
#     vectorizer2.fit(x_train["text"])

#     x_train_bow_2 = vectorizer2.transform(x_train['text'])
#     x_dev_bow_2 = vectorizer2.transform(x_dev["text"])

#     #Regression with N-gram
#     model_logistic_Ngram = LogisticRegression(penalty='l2')
#     model_logistic_Ngram.fit(x_train_bow_2, y_train)

#     y_dev_predict_Ngram = model_logistic_Ngram.predict(x_dev_bow_2)

#     if f1_score(y_dev, y_dev_predict_Ngram) > max_score: 
#         max_score = f1_score(y_dev, y_dev_predict_Ngram)
#         M = i

# print("Optimum M for Ngram is: " + str(M))

### (i)

In [ ]:
text = token_to_sentence(train)
train['text'] = text
y_train = train.iloc[:,-1:]

vectorizer.fit(train["text"])
test_bow = vectorizer.transform(test['text'])
x_train_bow_3 = vectorizer.transform(train["text"])
test_bow = vectorizer.transform(test["text"])

model_logistic_L2 = LogisticRegression(penalty='l2')
model_logistic_L2.fit(x_train_bow_3, y_train)

y_train_predict = model_logistic_L2.predict(test_bow)

with open("submission.csv","w") as f:
    writer = csv.writer(f)
    row = ["id", "target"]
    writer.writerow(row)
    for i in range(len(y_train_predict)):
        row = [test['id'][i], y_train_predict[i]]
        writer.writerow(row)

#Kaggle F1-score: 0.79436
#This is higher than I expected since the model is performing better on the testing data than the dev data.
#It could be that now the training data is about 43% larger because it includes the original development data, therefore,
#the model is probably better than before
